<a href="https://colab.research.google.com/github/SuperPanww/courses_ML20/blob/master/hw1_panww.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import pandas as pd
import numpy as np
from google.colab import drive 
!gdown --id '1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm' --output data.zip
!unzip data.zip
# data = pd.read_csv('gdrive/My Drive/hw1-regression/train.csv', header = None, encoding = 'big5')
data = pd.read_csv('./train.csv', encoding = 'big5')

Downloading...
From: https://drive.google.com/uc?id=1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm
To: /content/data.zip
100% 177k/177k [00:00<00:00, 81.4MB/s]
Archive:  data.zip
  inflating: test.csv                
  inflating: train.csv               


In [2]:
data.head(18)

,日期,測站,測項,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,12,15,17,20,22,22,22,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,0.47,0.78,0.74,0.59,0.52,0.41,0.4,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,0.13,0.26,0.23,0.2,0.18,0.12,0.11,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,2.2,6.6,7.9,4.2,2.9,3.4,3,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,2014/1/1,豐原,NO2,16,9.2,8.2,6.9,6.8,3.8,6.9,7.8,15,21,14,11,14,12,11,11,22,28,19,12,8.1,7,6.9,6
6,2014/1/1,豐原,NOx,17,9.8,8.7,8.6,8.5,5.3,8.8,9.9,22,29,18,14,17,15,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,2014/1/1,豐原,O3,16,30,27,23,24,28,24,22,21,29,44,58,50,57,65,64,51,34,33,34,37,38,38,36
8,2014/1/1,豐原,PM10,56,50,48,35,25,12,4,2,11,38,56,64,56,57,52,51,66,85,85,63,46,36,42,42
9,2014/1/1,豐原,PM2.5,26,39,36,35,31,28,25,20,19,30,41,44,33,37,36,45,42,49,45,44,41,30,24,13


In [0]:
data = data.iloc[:,3:]
data[data == 'NR' ] = 0
raw_data = data.to_numpy()

In [0]:
month_data = {}
for month in range(12):
  sample = np.empty([18,480])
  for day in range(20):
    sample[:,day*24:(day+1)*24] = raw_data[18*(month*20+day):18*(month*20+1+day),:]
  month_data[month] = sample

In [5]:
x = np.empty([12*471,18*9],dtype=float)
y = np.empty([12*471,1],dtype=float)
for month in range(12):
  for day in range(20):
    for hour in range(24):
      if day == 19 and hour > 14:
        continue
      x[month*471+day*24+hour,:] = month_data[month][:,day*24+hour:day*24+hour+9].reshape(1,-1)
      y[month*471+day*24+hour,0] = month_data[month][9,day*24+hour+9]
print(x)
print(y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


In [0]:
mean_x = np.mean(x,axis=0)
std_x = np.std(x,axis=0)
for i in range(len(x)):
  for j in range(len(x[0])):
    if std_x[j] != 0:
      x[i][j] = (x[i][j] - mean_x[j])/std_x[j]


In [7]:
import math
x_train_set = x[:math.floor(len(x)*0.8),:]
y_train_set = y[:math.floor(len(y)*0.8),:]
x_validation = x[math.floor(len(x)*0.8):,:]
y_validation = y[math.floor(len(x)*0.8):,:]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.3895055

In [8]:
dim = 18*9+1
w = np.zeros([dim,1])
x_set = np.concatenate((np.ones([len(x_train_set),1]),x_train_set),axis = 1).astype(float)
learning_rate = 100
iter_time = 1000
adagrad = np.zeros([dim,1])
eps = 0.0000000001
for i_th in range(iter_time):
  loss = np.square(np.sum(np.power((np.dot(x_set,w) - y_train_set),2))/len(x_train_set))
  if (i_th%100 == 0):
    print(str(i_th)+':'+str(loss))
  gradient = 2 * np.dot(x_set.transpose(),np.dot(x_set,w)-y_train_set)
  adagrad += gradient ** 2
  w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy',w)


0:550557.106856347
100:266647.3476944906
200:11824.048874576361
300:3399.5686402534056
400:2201.5650393179785
500:1805.5464199042285
600:1601.1010199382633
700:1472.2565219837334
800:1383.7500533155865
900:1320.2201532078861


In [9]:
testdata = pd.read_csv('./test.csv',header = None , encoding = 'big5')
test_data = testdata.iloc[:,2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240,18*9],dtype = float)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [0]:
for i in range(240):
  test_x[i,:] = test_data[i*18:(i+1)*18,:].reshape(1,-1)
for i in range(len(test_x)):
  for j in range(len(test_x[0])):
    if std_x[j] != 0:
      test_x[i][j] = (test_x[i][j] - mean_x[j])/std_x[j]
test_x = np.concatenate((np.ones([240,1]),test_x),axis = 1).astype(float)



In [0]:
w = np.load('weight.npy')
ans_y = np.dot(test_x,w)
ans_y

In [0]:
import csv
with open('submit.csv',mode='w',newline='') as submit_file:
  csv_writer = csv.writer(submit_file)
  header = ['id','value']
  print(header)
  csv_writer.writerow(header)
  for i in range(240):
    row = ['id_'+str(i),ans_y[i][0]]
    csv_writer.writerow(row)
    print(row)

In [15]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Create & upload a text file.
#你想要导出的文件的名字
uploaded = drive.CreateFile({'title': 'submit.csv'})
#改为之前生成文件的名字
uploaded.SetContentFile('submit.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1XhOUBiT-_1hJddXgvzkJ4W2xEkFGSVxj
